# **Unificação de dados de Dados de Acidentes de Trânsito no Brasil (2007-2024)**

Este notebook descreve o processo de de pré-processamento dos dados coletados  sobre acidentes de trânsito no Brasil, utilizando informações fornecidas pelo Departamento Nacional de Trânsito (DETRAN).

In [2]:
# Estabelecendo o diretório raiz do projeto
notebook_path = Path.cwd()
project_root = notebook_path.parent
sys.path.append(str(project_root / "src"))

# Importando bibliotecas necessárias
import sys
import os
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from config.config_project import ConfigProject
from preprocessing.merge_datasets import DatasetMerger
from preprocessing.file_read_pandas import PandasReadFile
from preprocessing.dataframe_manipulation import DataFrameManipulation

# Carregando a configuração do projeto
config = ConfigProject()



# Exibindo a configuração do projeto
config_info = {
    "Diretórios de Entrada": config.get("paths.save_files"),
    "Diretórios de Saída": config.get("paths.output_files")
}
config_info


{'Diretórios de Entrada': './files/raw/',
 'Diretórios de Saída': './files/processed/'}

### Explicação da Configuração do Projeto
Antes de começar a manipulação dos dados, é importante garantir que o módulo está configurado corretamente para localizar os arquivos de entrada e saída. A configuração é carregada através da classe `ConfigProject`, que busca os diretórios definidos no arquivo de configuração.

In [3]:
# Exibindo a configuração carregada do arquivo
config_info


{'Diretórios de Entrada': './files/raw/',
 'Diretórios de Saída': './files/processed/'}

## **Leitura dos Arquivos CSV**
Agora, vamos verificar os arquivos CSV disponíveis no diretório de entrada. Usaremos a classe `PandasReadFile` para ler esses arquivos.

In [4]:
# Inicializando a classe para ler arquivos
merger = DatasetMerger()

# Verificando os arquivos CSV disponíveis para os anos de interesse
available_files = sorted(Path(merger.data_dir).glob('datatran*.csv'))
print("Arquivos encontrados:")
for file in available_files:
    print(f"- {file.name}")


Arquivos encontrados:
- datatran2007.csv
- datatran2008.csv
- datatran2009.csv
- datatran2010.csv
- datatran2011.csv
- datatran2012.csv
- datatran2013.csv
- datatran2014.csv
- datatran2015.csv
- datatran2016.csv
- datatran2017.csv
- datatran2018.csv
- datatran2019.csv
- datatran2020.csv
- datatran2021.csv
- datatran2022.csv
- datatran2023.csv
- datatran2024.csv


## **Analisando um Arquivo Específico**
Vamos analisar um arquivo específico para entender como os dados estão estruturados.

In [5]:
# Função para analisar o dataset
def analyze_dataset(file_path):
    print(f"\nAnalisando {file_path.name}:")
    df = PandasReadFile.read_csv_file(file_path)

    # Exibindo informações sobre o DataFrame
    print(f"\nDimensões: {df.shape}")
    print(f"\nColunas presentes:")
    for col in df.columns:
        print(f"- {col}")

    print(f"\nPrimeiras linhas:")
    display(df.head())

    return df

# Analisando o primeiro e último arquivo
oldest_file = min(available_files)
newest_file = max(available_files)

df_old = analyze_dataset(oldest_file)
df_new = analyze_dataset(newest_file)



Analisando datatran2007.csv:

Dimensões: (127675, 26)

Colunas presentes:
- id
- data_inversa
- dia_semana
- horario
- uf
- br
- km
- municipio
- causa_acidente
- tipo_acidente
- classificacao_acidente
- fase_dia
- sentido_via
- condicao_metereologica
- tipo_pista
- tracado_via
- uso_solo
- ano
- pessoas
- mortos
- feridos_leves
- feridos_graves
- ilesos
- ignorados
- feridos
- veiculos

Primeiras linhas:


c:\Users\joseb\OneDrive\Documentos\GitHub\RoadSafeAI\src\preprocessing\file_read_pandas.py:15: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, encoding='cp1252', sep=';')


,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,uso_solo,ano,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos
0,10,11/06/2007,Segunda,15:30:00,MG,381,623.2,OLIVEIRA,Falta de atenção,Colisão frontal,...,Rural,2007,9,0,4,0,5,0,4,6
1,10,11/06/2007,Segunda,15:30:00,MG,381,623.2,OLIVEIRA,Falta de atenção,Colisão frontal,...,Rural,2007,9,0,4,0,5,0,4,6
2,1032898,13/08/2007,Segunda,14:25:00,MG,40,585.5,ITABIRITO,Outras,Saída de Pista,...,Rural,2007,3,0,0,1,2,0,1,1
3,1051130,12/02/2007,Segunda,02:10:00,MA,135,11.0,SAO LUIS,Animais na Pista,Atropelamento de animal,...,Urbano,2007,5,2,2,1,0,0,3,1
4,1066824,20/11/2007,Terça,05:30:00,CE,222,30.8,CAUCAIA,Defeito mecânico em veículo,Capotamento,...,Rural,2007,1,0,1,0,0,0,1,1



Analisando datatran2024.csv:

Dimensões: (73114, 30)

Colunas presentes:
- id
- data_inversa
- dia_semana
- horario
- uf
- br
- km
- municipio
- causa_acidente
- tipo_acidente
- classificacao_acidente
- fase_dia
- sentido_via
- condicao_metereologica
- tipo_pista
- tracado_via
- uso_solo
- pessoas
- mortos
- feridos_leves
- feridos_graves
- ilesos
- ignorados
- feridos
- veiculos
- latitude
- longitude
- regional
- delegacia
- uop

Primeiras linhas:


,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,feridos_graves,ilesos,ignorados,feridos,veiculos,latitude,longitude,regional,delegacia,uop
0,571789.0,2024-01-01,segunda-feira,03:56:00,ES,101,38,CONCEICAO DA BARRA,Ultrapassagem Indevida,Colisão lateral sentido oposto,...,1,1,1,1,3,"-18,48261","-39,92379",SPRF-ES,DEL04-ES,UOP02-DEL04-ES
1,571804.0,2024-01-01,segunda-feira,04:50:00,PI,343,185,PIRIPIRI,Manobra de mudança de faixa,Colisão frontal,...,0,1,0,0,2,"-4,29603281","-41,76732659",SPRF-PI,DEL02-PI,UOP01-DEL02-PI
2,571806.0,2024-01-01,segunda-feira,04:30:00,BA,116,578,BREJOES,Ingestão de álcool pelo condutor,Colisão frontal,...,0,1,2,0,4,"-13,07158302","-39,9611107",SPRF-BA,DEL03-BA,UOP02-DEL03-BA
3,571818.0,2024-01-01,segunda-feira,06:30:00,SE,101,18,MALHADA DOS BOIS,Reação tardia ou ineficiente do condutor,Saída de leito carroçável,...,1,0,2,1,3,"-10,35601949","-36,90552235",SPRF-SE,DEL02-SE,UOP02-DEL02-SE
4,571838.0,2024-01-01,segunda-feira,05:00:00,MT,364,240,RONDONOPOLIS,Condutor deixou de manter distância do veículo...,Colisão traseira,...,0,2,1,0,3,"-16,17914141","-54,78905337",SPRF-MT,DEL02-MT,UOP01-DEL02-MT


Aqui, usamos a função **`analyze_dataset`** para carregar e mostrar os detalhes do primeiro e do último arquivo CSV encontrado no diretório. Exibimos as dimensões do DataFrame, as colunas presentes e as primeiras linhas para uma visualização inicial dos dados.

## **Unificação dos Dados**
Agora vamos começar o processo de unificação dos datasets. Usaremos a classe `**DatasetMerger**` para unir os dados de diferentes anos.

In [6]:
# Definindo os parâmetros para a unificação
start_year = 2007
end_year = 2024
include_extra_columns = False

# Unificando os datasets para o período definido
print("Iniciando processo de unificação...")
merged_df = merger._merge_datasets(start_year, end_year, include_extra_columns)

# Exibindo as dimensões do dataset unificado
print(f"\nDataset unificado criado com sucesso!")
print(f"Dimensões finais: {merged_df.shape}")


2025-01-30 16:53:55,611 - preprocessing.merge_datasets - INFO - 18 arquivos CSV encontrados para o período 2007-2024.
2025-01-30 16:53:55,612 - preprocessing.merge_datasets - INFO - Processando dataset do ano 2007: datatran2007.csv


Iniciando processo de unificação...
FileName: datatran2007.csv
FileName: datatran2008.csv
FileName: datatran2009.csv
FileName: datatran2010.csv
FileName: datatran2011.csv
FileName: datatran2012.csv
FileName: datatran2013.csv
FileName: datatran2014.csv
FileName: datatran2015.csv
FileName: datatran2016.csv
FileName: datatran2017.csv
FileName: datatran2018.csv
FileName: datatran2019.csv
FileName: datatran2020.csv
FileName: datatran2021.csv
FileName: datatran2022.csv
FileName: datatran2023.csv
FileName: datatran2024.csv
FileName: datatran2007.csv


c:\Users\joseb\OneDrive\Documentos\GitHub\RoadSafeAI\src\preprocessing\file_read_pandas.py:15: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, encoding='cp1252', sep=';')
2025-01-30 16:53:56,143 - preprocessing.merge_datasets - INFO - Processando dataset do ano 2008: datatran2008.csv


FileName: datatran2008.csv


c:\Users\joseb\OneDrive\Documentos\GitHub\RoadSafeAI\src\preprocessing\file_read_pandas.py:15: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  return pd.read_csv(file_path, encoding='cp1252', sep=';')
2025-01-30 16:53:56,717 - preprocessing.merge_datasets - INFO - Processando dataset do ano 2009: datatran2009.csv


...

FileName: datatran2022.csv


2025-01-30 16:54:03,304 - preprocessing.merge_datasets - INFO - Processando dataset do ano 2023: datatran2023.csv


FileName: datatran2023.csv


2025-01-30 16:54:03,752 - preprocessing.merge_datasets - INFO - Processando dataset do ano 2024: datatran2024.csv


FileName: datatran2024.csv

Dataset unificado criado com sucesso!
Dimensões finais: (2122296, 25)


A unificação dos dados é feita a partir de arquivos CSV de diferentes anos. A função **`_merge_datasets`** percorre os arquivos de entrada, processa cada dataset e os concatena em um único DataFrame. Dependendo da configuração, as colunas adicionais (como latitude e longitude) podem ser incluídas.

## **Salvando o Dataset Unificado**
Após a unificação dos dados, salvamos o dataset final em um arquivo CSV. Também geramos um arquivo de metadados que descreve o dataset unificado.

In [7]:
# Salvando o dataset unificado
merger.save_merged_dataset(merged_df)
print("Dataset salvo com sucesso!")

# Verificando o tamanho do arquivo gerado
output_file = Path(merger.output_dir) / 'datatran_merged.csv'
file_size_mb = output_file.stat().st_size / (1024 * 1024)
print(f"\nTamanho do arquivo: {file_size_mb:.2f} MB")


2025-01-30 16:54:33,394 - preprocessing.merge_datasets - INFO - Dataset unificado salvo em C:\Users\joseb\OneDrive\Documentos\GitHub\RoadSafeAI\files\processed\datatran_merged.csv
2025-01-30 16:54:33,406 - preprocessing.merge_datasets - INFO - Metadados salvos em C:\Users\joseb\OneDrive\Documentos\GitHub\RoadSafeAI\files\processed\metadata.txt


Dataset salvo com sucesso!

Tamanho do arquivo: 380.58 MB


## **Exibindo o Dataset Final**
Agora que o dataset foi salvo com sucesso, vamos carregar o arquivo e exibir as primeiras linhas para garantir que tudo esteja correto.

In [8]:
# Carregando o arquivo salvo
df_merged = pd.read_csv(output_file, sep=";")
print("\nPrimeiras linhas do dataset unificado:")
display(df_merged.head())



Primeiras linhas do dataset unificado:


,id,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,...,tracado_via,uso_solo,pessoas,mortos,feridos_leves,feridos_graves,ilesos,ignorados,feridos,veiculos
0,175827.0,2007-01-01,Segunda,17:30:00,MG,381.0,485.0,BETIM,Falta de atenção,Colisão traseira,...,Cruzamento,Urbano,2,0,0,0,2,0,0,2
1,174220.0,2007-01-01,Segunda,14:15:00,RJ,40.0,43.9,AREAL,Velocidade incompatível,Saída de Pista,...,Curva,Rural,1,0,0,0,1,0,0,1
2,175540.0,2007-01-01,Segunda,16:00:00,PE,101.0,32.0,IGARASSU,Não guardar distância de segurança,Colisão lateral,...,Reta,Rural,2,0,0,0,2,0,0,2
3,175544.0,2007-01-01,Segunda,12:00:00,MG,50.0,0.2,ARAGUARI,Outras,Saída de Pista,...,Curva,Rural,4,0,3,1,0,0,4,1
4,175545.0,2007-01-01,Segunda,08:40:00,MG,381.0,397.4,NOVA UNIAO,Outras,Saída de Pista,...,Curva,Rural,1,0,0,0,1,0,0,1


# **Metadados do Dataset**
Junto com o arquivo CSV, também geramos um arquivo de metadados contendo informações sobre o dataset, como o número de registros, o período de dados e a última atualização.

In [9]:
# Exibindo o conteúdo dos metadados
metadata_path = Path(merger.output_dir) / 'metadata.txt'
with open(metadata_path, 'r', encoding='utf-8') as f:
    metadata = f.read()

print("\nMetadados do Dataset:")
print(metadata)



Metadados do Dataset:
total_registros: 2122296
periodo: 2007-01-01 a 2024-12-31
ultima_atualizacao: 2025-01-30 16:54:33
colunas: ['id', 'data_inversa', 'dia_semana', 'horario', 'uf', 'br', 'km', 'municipio', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'fase_dia', 'sentido_via', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'uso_solo', 'pessoas', 'mortos', 'feridos_leves', 'feridos_graves', 'ilesos', 'ignorados', 'feridos', 'veiculos']



Este notebook ofereceu uma explicação passo a passo do módulo de unificação de dados e fornece uma interface interativa para visualização dos resultados, tornando o processo mais transparente e fácil de acompanhar.